Note: all ".to(device)" are deleted because Apple M1 chip does does support it.

Would recommend using Google Colab instead because it will be much faster

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#env" data-toc-modified-id="env-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>env</a></span><ul class="toc-item"><li><span><a href="#d22:-no-noise;-fixed-wgain" data-toc-modified-id="d22:-no-noise;-fixed-wgain-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>d22: no noise; fixed wgain</a></span></li></ul></li><li><span><a href="#make-LSTM-agent" data-toc-modified-id="make-LSTM-agent-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>make LSTM agent</a></span></li><li><span><a href="#TRAIN" data-toc-modified-id="TRAIN-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TRAIN</a></span></li><li><span><a href="#Test" data-toc-modified-id="Test-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test</a></span><ul class="toc-item"><li><span><a href="#test-and-save-variables" data-toc-modified-id="test-and-save-variables-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>test and save variables</a></span></li></ul></li><li><span><a href="#Animation" data-toc-modified-id="Animation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Animation</a></span></li></ul></div>

In [11]:
from LSTM_functions import*

import os
import torch
import numpy as np
import torch.nn.functional as F
import random
import pickle
import gym
from gym import spaces, Env
from gym.spaces import Dict, Box
import torch
from numpy import pi
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import math
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.linalg import vector_norm
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)

device = "mps" if torch.backends.mps.is_available() else "cpu"

# env

## d22: no noise; fixed wgain 

In [29]:
model_path = "LSTM_data/LSTM_July_26_2"

## Reward: 12480 after training for 3000 + episodes

## Reward: 12100 for e3 env. Lol....not much of a difference

In [13]:
gamma= 0.995
soft_q_lr = 0.0015
policy_lr = 0.003  
alpha_lr = 0.002 
update_itr= 1
hidden_dim= 128
reward_scale= 10
target_entropy= -2
soft_tau= 0.015


# obs = [angle, distance, memory, angle2, distance2, memory2, ...]
# before = [angle, angle2 ..., distance, distance2, ... memory, memory2]


reward_per_episode = 0

class MultiFF(Env):
  def __init__(self):
      super(MultiFF, self).__init__()
      self.num_ff = 200
      self.arena_radius = 1000
      self.episode_len = 1024 
      self.dt = 0.25 
      #self.current_episode = 0
      #self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.action_space = spaces.Box(low=-1., high=1., shape=(2,),dtype=np.float32)
      self.obs_ff = 2
      self.observation_space = spaces.Box(low=-1., high=1., shape=(self.obs_ff*3,),dtype=np.float32)
      self.terminal_vel = 0.01
      #self.pro_noise_std = 0.005
      self.vgain = 200
      self.wgain = pi/4
      self.reward_per_ff = 100
      #self.time_cost = 0.005
      #self.total_time = 0
      #self.zero_action = False
      #self.target_update_counter_num = 20
      #self.reward_per_episode = []
      #self.update_slots = True
      #self.closest_ff_distance = 200
      self.pro_noise_std = 0.005
      self.epi_num = -1
      self.has_sped_up_before = False
      self.invisible_distance = 400
      self.invisible_angle = 2*pi/9
      self.reward_boundary = 25
      # self.dev_v_cost = 1
      self.flash_on_interval = 2.1
      # self.distance_noise_factor = 1/6
      # self.distance_noise_constant = 5
      # self.angle_noise_factor = 1/8
      # self.angle_noise_constant = 1/8
      self.num_intervals = 1500
      self.distance2center_cost = 2
      






  def reset(self):
    self.epi_num += 1
    print("\n episode: ", self.epi_num)
    self.num_targets = 0
    #self.past_speeds = []
    self.time = 0
    #self.counter = 0
    #self.current_target_index = torch.tensor([999], dtype=torch.int32)
    #self.previous_target_index = self.current_target_index
    self.ff_flash = []
    self.has_sped_up_before = False
    

    if self.flash_on_interval > 0.31:
      global reward_per_episode
      if reward_per_episode >= 1000:
        self.flash_on_interval-=0.3 
        reward_per_episode = 800
        global best_cum_rewards
        best_cum_rewards = best_cum_rewards/2
    else:
      self.distance2center_cost = 0
    print("flash_on_interval: ",   self.flash_on_interval)

    for i in range(self.num_ff):
      
      first_flash = torch.rand(1)
      intervals = torch.poisson(torch.ones(self.num_intervals-1)*3)
      t0 = torch.cat((first_flash, first_flash+torch.cumsum(intervals, dim=0)+torch.cumsum(torch.ones(self.num_intervals-1)*0.3, dim=0)))
      t1 = t0 + torch.ones(self.num_intervals)*self.flash_on_interval
      self.ff_flash.append(torch.stack((t0, t1), dim=1))

    self.ffr = torch.sqrt(torch.rand(self.num_ff))*self.arena_radius # The radius of the arena changed from 1000 cm/s to 200 cm/s
    self.fftheta = torch.rand(self.num_ff)*2*pi
    #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
    self.ffx = torch.cos(self.fftheta) * self.ffr
    self.ffy = torch.sin(self.fftheta) * self.ffr
    self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
    self.ffx2 = self.ffx.clone()
    self.ffy2 = self.ffy.clone()
    self.ffxy2 = torch.stack((self.ffx2, self.ffy2), dim=1)
    #self.ff_info={}
    #self.update_slots = True
    self.agentx = torch.tensor([0])
    self.agenty = torch.tensor([0])
    self.agentr = torch.zeros(1)
    self.agentxy = torch.tensor([0, 0])
    self.agentheading = torch.zeros(1).uniform_(0, 2*pi)
    self.dv = torch.zeros(1).uniform_(-0.05, 0.05) 
    self.dw = torch.zeros(1)
    self.end_episode = False 
    self.obs = self.beliefs().numpy()
    #self.chunk_50s = 1
    self.episode_reward = 0
    #self.stop_rewarding_speed = False
    #self.current_obs_steps = 0
    return self.obs






  def calculate_reward(self):
      #action_cost=((self.previous_action[1]-self.action[1])**2+(self.previous_action[0]-self.action[0])**2)*self.mag_cost
      # To incorporate action_cost, we need to incorporate previous_action into decision_info
      #self.total_time += 1
      # In addition to rewarding the monkey for capturing the firefly, we also use different phases of rewards to teach monkey specific behaviours
      #reward = -self.time_cost
      reward = 0
      # Reward shaping
      # Phase I: reward the agent for learning to stop 
      # Always: reward the agent for capturing fireflies
      self.num_targets = 0
      if abs(self.sys_vel[1]) <= self.terminal_vel:
        captured_ff_index0 = (self.ff_distance_all <= self.reward_boundary).nonzero().reshape(-1)
        total_deviated_distance = torch.sum(self.ff_distance_all[captured_ff_index0]).item()
        captured_ff_index = captured_ff_index0.tolist()
        self.captured_ff_index = captured_ff_index
        num_targets = len(captured_ff_index)
        self.num_targets = num_targets
        if num_targets > 0: # If the monkey hs captured at least 1 ff
          # Calculate reward
          reward = reward + self.reward_per_ff * num_targets - total_deviated_distance*self.distance2center_cost 
          # Replace the captured ffs with ffs of new locations
          self.ffr[captured_ff_index]= torch.sqrt(torch.rand(num_targets))*self.arena_radius
          self.fftheta[captured_ff_index]= torch.rand(num_targets)*2*pi
          #self.ffrt = torch.stack((self.ffr, self.fftheta), dim=1) 
          self.ffx[captured_ff_index] = torch.cos(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffy[captured_ff_index] = torch.sin(self.fftheta[captured_ff_index]) * self.ffr[captured_ff_index]
          self.ffxy = torch.stack((self.ffx, self.ffy), dim=1)
          # Delete the information from self.ff_info
          #[self.ff_info.pop(key) for key in captured_ff_index if (key in self.ff_info)]
          #self.current_target_index = torch.tensor([999], dtype=torch.int32)
          #self.previous_target_index = self.current_target_index
          # Reward the firefly based on the average speed before capturing the firefly
          ##reward = reward + sum(self.past_speeds)/len(self.past_speeds)
          #print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "obs: ", list(np.round(self.obs, decimals = 2)), "n_targets: ",  num_targets)
          print(round(self.time, 2), "sys_vel: ", [round(i, 4) for i in self.sys_vel.tolist()], "n_targets: ",  num_targets)
          #self.update_slots = True
          #self.stop_rewarding_speed = True
        #elif self.has_sped_up_before == True:
        # based on Ruiyi's formula, using the distance of the closest ff in obs
        #reward += math.exp(-((self.ff_current[1, 0]**2)*(25/1.5)**2)/2)
        #self.has_sped_up_before = False
      self.prev_action = self.action.clone()

      return reward







  def step(self, action):
    self.time += self.dt
    action=torch.tensor(action)
    self.action = action.clone()
    action[1] = action[1]/2+0.5
    self.sys_vel=action.clone()
    self.state_step(action)
    self.obs = self.beliefs().numpy()
    reward=self.calculate_reward()
    self.episode_reward += reward
    
    if self.time >= self.episode_len*self.dt:
      self.end_episode = True
      #self.current_episode += 1
      print("Reward for the episode: ", self.episode_reward)
    #print("action: ", torch.round(action, decimals = 3), "obs: ", np.round(self.obs, decimals = 3), "Reward: ", round(reward, 3))
    return self.obs, reward, self.end_episode, {}




  def state_step(self,action):
    #vnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std 
    #wnoise=torch.distributions.Normal(0,torch.ones([1,1])).sample()*self.pro_noise_std
    vnoise = 0
    wnoise = 0
    self.dw_normed = (action[0]+wnoise)
    self.dv_normed = (action[1]+vnoise)
    self.dw = (action[0]+wnoise)*self.wgain*self.dt
    self.agentheading = self.agentheading + self.dw.item()
    self.dv =  (action[1]+vnoise)*self.vgain*self.dt
    self.dx= torch.cos(self.agentheading)*self.dv
    self.dy= torch.sin(self.agentheading)*self.dv
    self.agentx = self.agentx + self.dx.item()
    self.agenty = self.agenty + self.dy.item()
    self.agentxy = torch.cat((self.agentx, self.agenty))
    self.agentr = vector_norm(self.agentxy)
    self.agenttheta = torch.atan2(self.agenty, self.agentx)  
                               
    if self.agentr >= self.arena_radius:
      self.agentr = 2*self.arena_radius-self.agentr
      self.agenttheta = self.agenttheta + pi
      self.agentx = (self.agentr*torch.cos(self.agenttheta)).reshape(1,)
      self.agenty = (self.agentr*torch.sin(self.agenttheta)).reshape(1,)
      self.agentxy = torch.cat((self.agentx, self.agenty))
      self.agentheading = self.agenttheta - pi
    while self.agentheading >= 2*pi:
      self.agentheading = self.agentheading - 2*pi
    while self.agentheading < 0:
      self.agentheading = self.agentheading + 2*pi   








  def beliefs(self): 


    # Make a tensor containing the relative distance of all fireflies to the agent
    self.ff_distance_all = vector_norm(self.ffxy - self.agentxy, dim=1)
    #distance_noise = torch.distributions.Normal(0,self.ff_distance_all*self.distance_noise_factor+ self.distance_noise_constant).sample() 
    #self.ff_distance_all = self.ff_distance_all + distance_noise
    # Make a tensor containing the relative (real) angle of all fireflies to the agent
    ffradians = torch.atan2(self.ffy-self.agenty, self.ffx-self.agentx)
    angle0 = ffradians - self.agentheading
    angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
    angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi

    #angle_noise = torch.distributions.Normal(0,torch.abs(angle0)*self.angle_noise_factor+ self.angle_noise_constant).sample() 
    #angle0 = angle0+angle_noise
    # Adjust the angle based on reward boundary
    angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip( self.ff_distance_all, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
    angle2 = torch.clip(angle1,0,pi)
    ff_angle_all = torch.sign(angle0)* angle2
    # Update the tensor containing the uncertainties of all fireflies to the agent
    visible_ff = torch.logical_and( self.ff_distance_all < self.invisible_distance, torch.abs(angle0) < self.invisible_angle)
    self.visible_ff_indices0 = visible_ff.nonzero().reshape(-1)
    for index in self.visible_ff_indices0:
      ff = self.ff_flash[index]
      if not torch.any(torch.logical_and(ff[:, 0] <= self.time, ff[:, 1] >= self.time)):
        visible_ff[index] = False
    self.visible_ff_indices = visible_ff.nonzero().reshape(-1)

    # Consider the case where there are fewer than self.obs_ff fireflies that are in memory
    if torch.numel(self.visible_ff_indices) >= self.obs_ff:
      # Rank the ff whose "memory" is creater than 0 based on distance
      sorted_indices =  torch.topk(-self.ff_distance_all[self.visible_ff_indices], self.obs_ff).indices
      self.topk_indices = self.visible_ff_indices[sorted_indices]
      self.ffxy_topk = self.ffxy[self.topk_indices]
      #self.ff_distance_topk = vector_norm(self.ffxy_topk - self.agentxy, dim=1)
      self.ff_distance_topk = self.ff_distance_all[self.topk_indices]
      self.ff_angle_topk_2 = angle0[self.topk_indices]
      ff_angle_topk_3 = ff_angle_all[self.topk_indices]

      # # Calculate relative angles 
      # ffradians = torch.atan2(self.ffxy_topk[:,1]-self.agenty, self.ffxy_topk[:,0]-self.agentx)
      # angle0 = ffradians - self.agentheading
      # angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      # angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      # self.ff_angle_topk_2 = angle0.clone()
     
      # # Calculate relative angles of all ffs based on reward boundaries
      # # Adjust the angle based on reward boundary
      # angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      # angle2 = torch.clip(angle1,0,pi)
      # ff_angle_topk_3 = torch.sign(angle0)* angle2
      
      
      # Concatenate distance, angle, and memory
      ff_array = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk), dim=0)
    

    elif torch.numel(self.visible_ff_indices) == 0:
      ff_array = torch.tensor([[0], [0], [self.invisible_distance]]).repeat([1, self.obs_ff])
      self.topk_indices = torch.tensor([])

    else:
      sorted_distance, sorted_indices = torch.sort(-self.ff_distance_all[self.visible_ff_indices])
      self.topk_indices = self.visible_ff_indices[sorted_indices]
      self.ffxy_topk = self.ffxy[self.topk_indices ]
      
      self.ff_distance_topk = self.ff_distance_all[self.topk_indices]
      self.ff_angle_topk_2 = angle0[self.topk_indices]
      ff_angle_topk_3 = ff_angle_all[self.topk_indices]
      # self.ff_distance_topk = vector_norm(self.ffxy_topk - self.agentxy, dim=1)
      # # Calculate relative angles 
      # ffradians = torch.atan2(self.ffxy_topk[:,1]-self.agenty, self.ffxy_topk[:,0]-self.agentx)
      # angle0 = ffradians - self.agentheading
      # angle0[angle0 > pi] = angle0[angle0 > pi] - 2*pi
      # angle0[angle0 < -pi] = angle0[angle0 < -pi] + 2*pi
      # self.ff_angle_topk_2 = angle0.clone()
      # # Calculate relative angles of all ffs based on reward boundaries
      # # Adjust the angle based on reward boundary
      # angle1 = torch.abs(angle0)-torch.abs(torch.arcsin(torch.div(self.reward_boundary, torch.clip(self.ff_distance_topk, self.reward_boundary, 400) ))) # use torch clip to get valid arcsin input
      # angle2 = torch.clip(angle1,0,pi)
      # ff_angle_topk_3 = torch.sign(angle0)* angle2
      
      # Concatenate distance, angle, and memory
      ff_array0 = torch.stack((self.ff_angle_topk_2, ff_angle_topk_3, self.ff_distance_topk), dim=0)
      needed_ff = self.obs_ff - torch.numel(self.visible_ff_indices)
      ff_array = torch.stack([ff_array0.reshape([3,-1]), torch.tensor([[0], [0], [self.invisible_distance]]).repeat([1, needed_ff])], dim=1)
      

    # ff_array[0:2,:] = ff_array[0:2,:]/pi
    # ff_array[2,:] = (ff_array[2,:]/self.invisible_distance-0.5)*2
    ff_array[2,:] = ff_array[2,:]/self.invisible_distance
    self.ff_array = ff_array.clone()
    return torch.flatten(ff_array.transpose(0, 1))


env = MultiFF()  
env.reset()  



 episode:  0
flash_on_interval:  2.1


array([0, 0, 1, 0, 0, 1])

# make LSTM agent

In [14]:
replay_buffer_size = 100
batch_size = 10
replay_buffer = ReplayBufferLSTM2(replay_buffer_size)

# choose env
env = MultiFF()
action_space = env.action_space
state_space  = env.observation_space
action_range=1.

action_dim = action_space.shape[0]

# hyper-parameters for RL training
max_episodes  = 1000
max_steps   = 1024
frame_idx   = 0
AUTO_ENTROPY=True
DETERMINISTIC=False
rewards     = []
train_freq = 100




sac_trainer=SAC_Trainer(replay_buffer, state_space, action_space, hidden_dim=hidden_dim, action_range=action_range,\
                          gamma = gamma, soft_q_lr = soft_q_lr, policy_lr = policy_lr, alpha_lr = alpha_lr, \
                          batch_size = batch_size, update_itr = update_itr, reward_scale = reward_scale, \
                        target_entropy = target_entropy, soft_tau = soft_tau, train_freq = train_freq)



/Users/dusiyi/.conda/envs/Multifirefly-Project/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Soft Q Network (1,2):  QNetworkLSTM2(
  (linear1): Linear(in_features=10, out_features=128, bias=True)
  (lstm1): LSTM(128, 128, dropout=0.2)
  (linear2): Linear(in_features=128, out_features=1, bias=True)
)
Policy Network:  SAC_PolicyNetworkLSTM(
  (linear1): Linear(in_features=6, out_features=128, bias=True)
  (linear2): Linear(in_features=8, out_features=128, bias=True)
  (lstm1): LSTM(128, 128, dropout=0.2)
  (linear3): Linear(in_features=256, out_features=128, bias=True)
  (mean_linear): Linear(in_features=128, out_features=2, bias=True)
  (log_std_linear): Linear(in_features=128, out_features=2, bias=True)
)


# TRAIN

In [ ]:
best_avg_rewards = -100

# training loop
#sac_trainer.load_model(model_path)
max_episodes  = 10000
eval_episodes = 2
best_cum_rewards = -1000000


sac_trainer.soft_q_net1.train()
sac_trainer.soft_q_net2.train()
sac_trainer.policy_net.train()

eval_counter = 0
eval_rewards = []

for eps in range(max_episodes):
    state =  env.reset()
    last_action = env.action_space.sample()
    episode_state = []
    episode_action = []
    episode_last_action = []
    episode_reward = []
    episode_next_state = []
    episode_done = []
    hidden_out = (torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device), \
        torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device))  # initialize hidden state for LSTM, (hidden, cell), each is (layer, batch, dim)             
    for step in range(max_steps):
        hidden_in = hidden_out

        action, hidden_out = sac_trainer.policy_net.get_action(state, last_action, hidden_in, deterministic = False)
        next_state, reward, done, _ = env.step(action)

            # env.render()       
            
        if step == 0:
            ini_hidden_in = hidden_in
            ini_hidden_out = hidden_out
        episode_state.append(state)
        episode_action.append(action)
        episode_last_action.append(last_action)
        episode_reward.append(reward)
        episode_next_state.append(next_state)
        episode_done.append(done) 

        state = next_state
        last_action = action
        
        if step % train_freq == 0:
          if len(replay_buffer) > batch_size:
              for i in range(update_itr):
                  _=sac_trainer.update(batch_size, reward_scale=10., auto_entropy=AUTO_ENTROPY, target_entropy=-1.*action_dim)

        if done:
            break
    replay_buffer.push(ini_hidden_in, ini_hidden_out, episode_state, episode_action, episode_last_action, \
        episode_reward, episode_next_state, episode_done)

    if eps % 15 == 0 and eps>0: # plot and model saving interval
        #np.save('rewards_LSTM', rewards)

        #evaluate
        eval_env = MultiFF()
        cum_rewards = 0        
        for eval_eps in range(eval_episodes):
            state =  env.reset()
            last_action = env.action_space.sample()
            hidden_out = (torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device), \
                torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device))  # initialize hidden state for LSTM, (hidden, cell), each is (layer, batch, dim)              
            for step in range(max_steps):
                hidden_in = hidden_out
                action, hidden_out = sac_trainer.policy_net.get_action(state, last_action, hidden_in, deterministic = DETERMINISTIC)
                next_state, reward, done, _ = env.step(action)
                cum_rewards = cum_rewards+reward
                state = next_state
                last_action = action
        reward_per_episode = cum_rewards/eval_episodes
        print("                      Best avg rewards:", best_avg_rewards, "Evaluation: rewards per episode", reward_per_episode)
        if cum_rewards > best_cum_rewards:
            best_cum_rewards = cum_rewards
            best_avg_rewards = cum_rewards/eval_episodes
            sac_trainer.save_model(model_path)
            path2 = os.path.join(model_path, 'buffer.pkl')
            with open(path2, 'wb') as f:
              pickle.dump(replay_buffer.buffer, f)
            print("                                            Best average rewards = ", best_avg_rewards)
            print("                                            Best model saved at episode "+ str(eps))
        eval_counter += 1
        eval_rewards.append(reward_per_episode)
        if (eval_counter > 1):
          plt.figure(figsize=(20,5))
          plt.plot(eval_rewards)
          plt.show()
          plt.close()
          if len(eval_rewards) > 50:
            eval_rewards = eval_rewards[-50:]
    print('Episode: ', eps, '| Episode Reward: ', np.sum(episode_reward))
    rewards.append(np.sum(episode_reward))


 episode:  18
flash_on_interval:  1.8
0.25 sys_vel:  [-0.9999, 0.0] n_targets:  1
6.75 sys_vel:  [1.0, 0.0001] n_targets:  1
14.25 sys_vel:  [-1.0, 0.0] n_targets:  1
17.25 sys_vel:  [-0.9461, 0.0] n_targets:  1
19.25 sys_vel:  [-0.9956, 0.0] n_targets:  1
19.75 sys_vel:  [-1.0, 0.0] n_targets:  1
33.0 sys_vel:  [-0.9758, 0.0] n_targets:  1
36.25 sys_vel:  [-0.6922, 0.0] n_targets:  1
42.25 sys_vel:  [-1.0, 0.0] n_targets:  1
59.5 sys_vel:  [0.6498, 0.0] n_targets:  1
68.0 sys_vel:  [0.6867, 0.0] n_targets:  1
90.0 sys_vel:  [-1.0, 0.0] n_targets:  1
96.75 sys_vel:  [-1.0, 0.0] n_targets:  1
130.75 sys_vel:  [0.9883, 0.0002] n_targets:  1
189.75 sys_vel:  [-1.0, 0.0] n_targets:  1
194.75 sys_vel:  [-1.0, 0.0] n_targets:  1
231.75 sys_vel:  [-1.0, 0.0009] n_targets:  1
235.75 sys_vel:  [-0.9966, 0.0008] n_targets:  1
238.25 sys_vel:  [-1.0, 0.0] n_targets:  1
254.25 sys_vel:  [0.9975, 0.0] n_targets:  1
Reward for the episode:  1306.4972229003906
Episode:  0 | Episode Reward:  1306.497

# Test

In [7]:
# Load buffer
path2 = os.path.join(model_path, 'buffer.pkl')
with open(path2, 'rb') as f:
  buffer_buffer = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/LSTM_data/July_26_2/buffer.pkl'

In [ ]:
env.flash_on_interval = 0.3
env.distance2center_cost = 0
#model_path = "/content/gdrive/MyDrive/fireflies_agent/lstm/July_18"
sac_trainer.load_model(model_path)
#env = MultiFF()
last_action = env.action_space.sample()
hidden_out = (torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device), \
        torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device))  # initialize hidden state for lstm, (hidden, cell), each is (layer, batch, dim)             

total_reward = 0

eps_num = 5
for eps in range(eps_num):

    state =  env.reset()
    episode_reward = 0
    hidden_out = (torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device), \
        torch.zeros([1, 1, hidden_dim], dtype=torch.float).to(device))  # initialize hidden state for lstm, (hidden, cell), each is (layer, batch, dim)
    
    for step in range(max_steps):
        hidden_in = hidden_out
        action, hidden_out = sac_trainer.policy_net.get_action(state, last_action, hidden_in, deterministic = DETERMINISTIC)

        next_state, reward, done, _ = env.step(action)   

        last_action = action
        episode_reward += reward
        state=next_state
    total_reward += episode_reward
    print('Episode: ', eps, '| Episode Reward: ', episode_reward)
print("Average reward per episode: ", total_reward/eps_num)

## test and save variables

In [ ]:
#model_path = "/LSTM_data/July_26_2" 
#sac_trainer.load_model(model_path)

# Test the trained agent
state =  env.reset()
last_action = env.action_space.sample()
hidden_out = (torch.zeros([1, 1, hidden_dim], dtype=torch.float)#  , \
        torch.zeros([1, 1, hidden_dim], dtype=torch.float)#  )  # initialize hidden state for lstm, (hidden, cell), each is (layer, batch, dim)             
episode_reward = 0
hidden_out = (torch.zeros([1, 1, hidden_dim], dtype=torch.float)#  , \
    torch.zeros([1, 1, hidden_dim], dtype=torch.float)#  )  # initialize hidden state for lstm, (hidden, cell), each is (layer, batch, dim)
n_steps = 700
mx = []
my = []
ffxy_all = []
ffxy_visible = []
ffxy2_all = []
time_rl = []
reward_log = []
mx_rewarded = []
my_rewarded = []
mheading = []
captured_ff = []
current_target = []
action_reward = []
num_targets = []
env_obs = []
#captured_ff_cum_x = []
#captured_ff_cum_y = []
all_captured_ff_x = []
all_captured_ff_y = []
visible_ff_indices_all = []
memory_ff_indices_all = []
obs_ff_indices_all = []



ff_angles2 = []
ff_distances2 = []

for step in range(n_steps):
  hidden_in = hidden_out
  action, hidden_out = sac_trainer.policy_net.get_action(state, last_action, hidden_in, deterministic = DETERMINISTIC)
  #print(step, "Action: ", action)
  previous_ffxy = env.ffxy
  next_state, reward, done, _ = env.step(action)  
  last_action = action
  episode_reward += reward
  state=next_state
  reward_log.append(reward)
  num_targets.append(env.num_targets)
  #print("reward: ", reward)
  #print(next_state)
  if env.num_targets > 0:
    print(step, "num_targets:",  env.num_targets)
    captured_ff.append(env.captured_ff_index)
    all_captured_ff_x = all_captured_ff_x + previous_ffxy[env.captured_ff_index][:,0].tolist()
    all_captured_ff_y = all_captured_ff_y + previous_ffxy[env.captured_ff_index][:,1].tolist()
  else:
    captured_ff.append(0)
  #captured_ff_cum_x.append(all_captured_ff_x)
  #captured_ff_cum_y.append(all_captured_ff_y)
  mx.append(env.agentx.item())
  my.append(env.agenty.item())
  mheading.append(env.agentheading.item())
  time_rl.append(env.time)
  ffxy_all.append(env.ffxy.clone())
  ffxy2_all.append(env.ffxy2.clone())
  ffxy_visible.append(env.ffxy[env.visible_ff_indices].clone())
  env_obs.append(obs)
  visible_ff_indices_all.append(env.visible_ff_indices)
  obs_ff_indices_all.append(env.topk_indices)
  if len(env.topk_indices) > 0:
    ff_angles2.append(env.ff_angle_topk_2)
    ff_distances2.append(env.ff_distance_topk)
  else:
    ff_angles2.append(torch.tensor([]))
    ff_distances2.append(torch.tensor([]))
  if done:
    obs = env.reset()
  #print(step, ffxy_visible[-1])



# Animation

In [ ]:
gif_dir = "" 
series = 10


# Animation 1

filename =  f"Demo{series + 1}"

start = 10
num_frame = 200
arena_radius = 1000
invisible_distance = 250
global fig; fig = plt.figure(dpi=100)
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['font.size'] = 15
colors_YlGn = plt.get_cmap("YlGn")(np.linspace(0,1,101))
#colors_RdPu = plt.get_cmap("RdPu")(np.linspace(0,1,101))
circle_theta = np.arange(0, 2*pi, 0.01)
circle_x = np.cos(circle_theta)*arena_radius
circle_y = np.sin(circle_theta)*arena_radius
fig, ax = plt.subplots()
cum_mx, cum_my = mx[start:start+num_frame], my[start:start+num_frame]
xmin, xmax = np.min(cum_mx), np.max(cum_mx)
ymin, ymax = np.min(cum_my), np.max(cum_my)
ax.set_xlim((xmin-invisible_distance, xmax+invisible_distance))
ax.set_ylim((ymin-invisible_distance, ymax+invisible_distance))
ax.set_aspect('equal')




def animate(j):
  ax.cla()
  ax.axis('off')
  ax.plot(circle_x, circle_y)
  i = j+start
  ax.scatter(mx[start:i+1],my[start:i+1], s=20)
  ax.scatter(ffxy_all[i].T[0], ffxy_all[i].T[1], alpha=0.9, c="gray", s=30)

  # if torch.numel(ffxy_visible[i]) > 0:
  #   ax.scatter(ffxy_visible[i].T[0], ffxy_visible[i].T[1], alpha=0.9, c="red", s=30)
 
  current_obs_ff = obs_ff_indices_all[i]
  if torch.numel(current_obs_ff) > 0:
    # Plot original positions of ff
    ax.scatter(ffxy2_all[i][current_obs_ff].T[0], ffxy2_all[i][current_obs_ff].T[1], s=60, alpha=0.5, color="green")
    # Plot position of ffs with noise
    ffx_recovered = torch.cos(ff_angles2[i]+mheading[i]) * ff_distances2[i] + mx[i]
    ffy_recovered = torch.sin(ff_angles2[i]+mheading[i]) * ff_distances2[i] + my[i]
    ax.scatter(ffx_recovered.clone(), ffy_recovered.clone(), alpha=0.9, c="black", s=30)
    # Plot the reward boundaries of the circles
    for z in range(len(current_obs_ff)):
      k = current_obs_ff[z]
      # real positions
      circle = plt.Circle((ffxy_all[i][k][0], ffxy_all[i][k][1]), 25, facecolor='yellow', edgecolor='orange', alpha=0.3, zorder=1)
      ax.add_patch(circle)
      # positions with noise
      circle = plt.Circle((ffx_recovered[z], ffy_recovered[z]), 25, facecolor='grey', edgecolor='orange', alpha=0.3, zorder=1)
      ax.add_patch(circle)
  
  # Add triangles
  ax.plot(np.array([mx[i], mx[i] + 30*np.cos(mheading[i]+2*pi/9)]), np.array([my[i] , my[i] + 30*np.sin(mheading[i]+2*pi/9)]), linewidth = 2)
  ax.plot(np.array([mx[i], mx[i] + 30*np.cos(mheading[i]-2*pi/9)]), np.array([my[i] , my[i] + 30*np.sin(mheading[i]-2*pi/9)]), linewidth = 2)
  


  # Plot ff positions recovered from relative angle and distance 
  ##if torch.numel(ffx_recovered) > 0: 
  ##  ffy_recovered = torch.sin(ff_angles2[i]+mheading[i]) * ff_distances2[i] + my[i]
  ##  ax.scatter(ffx_recovered.clone(), ffy_recovered.clone(), alpha=0.9, c="black", s=30)

  # Plot captured fireflies
  if num_targets[i] > 0:
    #ax.scatter(ffxy_all[i-1][captured_ff[i]].T[0], ffxy_all[i-1][captured_ff[i]].T[1], s=70, alpha=0.7, color="purple")
    #captured_ff_cum_x = captured_ff_cum_x+ffxy_all[i-1][captured_ff[i]].T[0].tolist()
    #captured_ff_cum_y = captured_ff_cum_y+ffxy_all[i-1][captured_ff[i]].T[1].tolist()
  #if len(captured_ff_cum_x) >0:
    ax.scatter(ffxy_all[i-1][captured_ff[i]].T[0], ffxy_all[i-1][captured_ff[i]].T[1], s=100, alpha=0.7, color="purple")
  ax.set_xlim((xmin-invisible_distance, xmax+invisible_distance))
  ax.set_ylim((ymin-invisible_distance, ymax+invisible_distance))
  ax.set_aspect('equal')

  #annotation = f"angle: {round(env_obs[i][0].item(), 2)}"
  #annotation = f"angle: {round(action_reward[i][0].item(), 2)}"
  #ax.text(0.78, 0.95, annotation, horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, fontsize=12, color="black", bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
anim = animation.FuncAnimation(fig, animate, 
                frames=num_frame, interval=500, repeat=True) 


os.makedirs(gif_dir, exist_ok=True)
#anim.save(f"{gif_dir}/{filename}.gif", writer='pillow', fps=60)

#anim.save('1-trial.gif', writer='pillow', fps=60)

writervideo = animation.FFMpegWriter(fps=4) 
anim.save(f"{gif_dir}/{filename}.mp4", writer=writervideo)
anim